<a href="https://colab.research.google.com/github/Klevertak/SensorPrediction/blob/dev/SensorPredict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd